In [13]:
import polars as pl
import duckdb
import utils_database as ud
import utils
import utils_model as um
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# ud.clear_sessions()
# ud.clear_messages()
# len(ud.list_sessions(user_id=4))

In [3]:
df_suppliers = pl.read_excel(source="data/master_data.xlsx",sheet_name='Suppliers')
df_materials = pl.read_excel(source="data/master_data.xlsx",sheet_name='Materials')
df_contracts = pl.read_excel(source="data/master_data.xlsx",sheet_name='Contracts')
df_sessions = pl.read_excel(source="data/master_data.xlsx",sheet_name='Sessions')

Could not determine dtype for column 7, falling back to string
Could not determine dtype for column 10, falling back to string


In [4]:
df_contracts.head()

contract_id,user_id,material_id,contract_details,price_per_unit,quantity,bundling_unit,bundling_amount,bundling_discount,payment_term,payment_term_markup,delivery_timeline,contract_period,contract_inflation,rebates_threshold_unit,rebates_discount,warranty,incoterms
i64,i64,i64,str,i64,i64,i64,str,i64,str,str,i64,i64,i64,i64,i64,i64,str
1,1,1,"""Unit Price: \$ 149;Quantity: 1…",149,14548,29096,null,4,"""NET60""",null,11,2,0,18766,2,1,"""FOB"""
2,1,2,"""Unit Price: \$ 104;Quantity: 1…",104,10144,20288,null,5,"""NET60""",null,17,2,1,13085,2,1,"""EXW"""
3,1,3,"""Unit Price: \$ 107;Quantity: 1…",107,17132,34264,null,7,"""NET60""",null,16,2,1,21072,3,1,"""CIF"""
4,1,4,"""Unit Price: \$ 109;Quantity: 1…",109,14532,29064,null,6,"""NET30""",null,11,2,1,17583,3,1,"""FOB"""
5,1,5,"""Unit Price: \$ 112;Quantity: 1…",112,11775,23550,null,6,"""NET30""",null,11,2,1,14601,3,1,"""FCA"""


In [4]:
# df_contracts = df_contracts.with_columns(
#     pl.struct([
#         'price_per_unit',
#         'quantity',
#         'bundling_unit',
#         'bundling_amount',
#         'bundling_discount',
#         'payment_term',
#         'payment_term_markup',
#         'delivery_timeline',
#         'contract_period',
#         'contract_inflation',
#         'rebates_threshold_unit',
#         'rebates_discount',
#         'warranty',
#         'incoterms'
#     ])
#     .map_elements(
#         lambda row: um.levers_to_offer_text(
#             price_per_unit = row['price_per_unit'],
#             quantity = row['quantity'],
#             bundling_unit = row['bundling_unit'],
#             bundling_amount = row['bundling_amount'],
#             bundling_discount = row['bundling_discount'],
#             payment_term = row['payment_term'],
#             payment_term_markup = row['payment_term_markup'],
#             delivery_timeline = row['delivery_timeline'],
#             contract_period = row['contract_period'],
#             contract_inflation = row['contract_inflation'],
#             rebates_threshold_unit = row['rebates_threshold_unit'],
#             rebates_discount = row['rebates_discount'],
#             warranty = row['warranty'],
#             incoterms = row['incoterms']
#         )
#     )
#     .alias('contract_details')
# )
# df_contracts.head()

In [5]:
df_materials.head(2)

material_id,description,quantity,price_per_unit,min_quantity,max_quantity,min_price_per_unit,max_price_per_unit
i64,str,i64,i64,i64,f64,i64,f64
1,"""Material 1""",14548,149,14548,18912.4,149,201.15
2,"""Material 2""",10144,104,10144,13187.2,104,140.4


In [18]:
df_materials = pl.read_excel(source="data/master_data.xlsx",sheet_name='Materials')
for material in df_materials.rows(named = True):
    material = ud.Material().load_from_data(material)
    ud.add_material(material=material)

ConstraintException: Constraint Error: Duplicate key "material_id: 1" violates primary key constraint. If this is an unexpected constraint violation please double check with the known index limitations section in our documentation (https://duckdb.org/docs/sql/indexes).

In [6]:
df_suppliers = pl.read_excel(source="data/master_data.xlsx",sheet_name='Suppliers')
for user in df_suppliers.rows(named = True):
    ud.add_user(username=user['username'],role=user['role'])

In [7]:
df_contracts = pl.read_excel(source="data/master_data.xlsx",sheet_name='Contracts')
for contract in df_contracts.rows(named = True):
    contract = ud.Contract().load_from_data(contract)
    ud.add_contract(contract=contract)

Could not determine dtype for column 7, falling back to string
Could not determine dtype for column 10, falling back to string


In [19]:
df_sessions = pl.read_excel(source="data/master_data.xlsx",sheet_name='Sessions')
for session in df_sessions.rows(named = True):
    session = ud.Session().load_from_data(session)
    ud.add_session(session=session)

In [32]:
con = duckdb.connect("chatbot.duckdb")
df = con.execute("""SELECT * FROM Users""").pl()
con.close()
df.head()

user_id,username,role,created_at
i32,str,str,datetime[μs]
1,"""accenture""","""admin""",2025-03-01 19:13:58.655
2,"""abc123""","""supplier""",2025-03-01 19:13:58.699
3,"""xyz321""","""supplier""",2025-03-01 19:13:58.754
4,"""cab 312""","""supplier""",2025-03-01 19:13:58.816
5,"""monojit""","""supplier""",2025-03-01 19:13:58.873


In [30]:
con = duckdb.connect("chatbot.duckdb")
df = con.execute("""SELECT ss.session_id, u.username,m.description, ss.start_time, ss.end_time 
        FROM Sessions ss
        LEFT JOIN Users u on ss.user_id = u.user_id
        LEFT JOIN Materials m on ss.material_id = m.material_id
        order by ss.start_time desc""").pl()
con.close()
df.head()

session_id,username,description,start_time,end_time
i32,str,str,datetime[μs],datetime[μs]
1,"""accenture""","""Material 2""",2025-03-01 00:00:00,2025-03-01 00:00:00
2,"""accenture""","""Material 3""",2025-03-01 00:00:00,2025-03-01 00:00:00
3,"""accenture""","""Material 4""",2025-03-01 00:00:00,2025-03-01 00:00:00
4,"""accenture""","""Material 1""",2025-03-01 00:00:00,2025-03-01 00:00:00
5,"""abc123""","""Material 8""",2025-03-01 00:00:00,2025-03-01 00:00:00


In [27]:
ud.pull_sessions_by_user('accenture')

session_id,username,description,start_time,end_time
i32,str,str,datetime[μs],datetime[μs]
4,"""accenture""","""Material 1""",2025-03-01 00:00:00,2025-03-01 00:00:00
1,"""accenture""","""Material 2""",2025-03-01 00:00:00,2025-03-01 00:00:00
2,"""accenture""","""Material 3""",2025-03-01 00:00:00,2025-03-01 00:00:00
3,"""accenture""","""Material 4""",2025-03-01 00:00:00,2025-03-01 00:00:00


In [14]:
material = ud.get_material_info(material_id = 2)
contract = ud.get_last_contract(user_id=1,material_id=2)
MIN_LEVERS = {
    'price_per_unit':material['min_price_per_unit'],
    'quantity': material['min_quantity']
}
MAX_LEVERS = {
    'price_per_unit':material['max_price_per_unit'],
    'quantity': material['max_quantity']
}
contract

{'contract_id': 2,
 'user_id': 1,
 'material_id': 2,
 'contract_details': None,
 'price_per_unit': 104.0,
 'quantity': 10144.0,
 'bundling_unit': 20288.0,
 'bundling_amount': None,
 'bundling_discount': 5.0,
 'payment_term': 'NET60',
 'delivery_timeline': 17.0,
 'contract_period': 2.0,
 'contract_inflation': 1.0,
 'rebates_threshold_unit': 13085.0,
 'rebates_discount': 2.0,
 'warranty': 1.0,
 'incoterms': 'EXW',
 'created_at': datetime.datetime(2025, 3, 1, 10, 6, 11, 296000),
 'expiry_tmstp': datetime.datetime(2025, 3, 1, 10, 6, 11, 296000)}

In [ ]:
conn = duckdb.connect('chatbot.duckdb')
matrls = ud.list_materials(user_id=1,db_connection=conn)
conn.close()

[(4,), (1,), (3,), (2,), (5,)]